In [291]:
import numpy as np
from PIL import Image, ImageDraw, ImageColor

In [292]:
class RotationTranslationMatrix:
    def __init__(self, m) -> None:
        assert m.shape[0] == m.shape[1]
        self.m = m

In [293]:
class Vector:
    def __init__(self, v):
        self.v = v

    def __eq__(self, __o):
        return np.all(self.v == __o.v)

    def __add__(self, __o):
        return Vector(self.v + __o.v)

    def __sub__(self, __o):
        return Vector(self.v - __o.v)

    def __mul__(self, __o):
        return Vector(self.v * __o)

    def norm(self):
        return np.linalg.norm(self.v)

    def unit(self):
        return self.v / self.norm()

In [294]:
class Point:
    def __init__(self, *xyz):
        self.xyz = np.array(xyz)

    def as_affine(self):
        self.xyz = self.xyz[:-1] / self.xyz[-1]

    def as_homogenious(self):
        self.xyz = np.append(self.xyz, 1) # return new?

    def to_vec(self):
        return Vector(self.xyz)

    def __eq__(self, __o):
        return np.all(self.xyz == __o.xyz)

    def __add__(self, __o: Vector):
        return Point(self.xyz + __o.v)

    def __sub__(self, __o):
        return Vector(self.xyz - __o.xyz)

    # def __mul__(self, __o):
    #     return Point(self.xyz * __o)

    # def __matmul__(self, m):
    #     return Vector(self.as_homogenious() @ m).to_affine()

    def __repr__(self) -> str:
        return f"Point {self.xyz}"

In [295]:
class Edge:
    def __init__(
        self,
        a,
        b,
    ):
        self.a = a
        self.b = b
        self.v = b - a

    def __eq__(self, __o):
        return (
            self.a == __o.a and self.b == __o.b or self.a == __o.b and self.b == __o.a
        )

    def normal(self, viewpoint):
        v_a = self.a - viewpoint
        v_b = self.b - self.a
        proj = np.dot(v_a, v_b) * v_b / v_b.norm() ** 2
        n = (v_a - proj).unit()
        return n

    def __repr__(self) -> str:
        return f"Edge <{self.a} {self.b}>"

In [296]:
# TODO
# 1) REDEFINE INITIALIZATION ✅
# 2) DEFINE ROTATION (CLOCKWISE / COUNTERCLOCKWISE)
# 3) DEFINE NORMALS FOR EDGES
# 4) DELAUNEY TRIANGUALTION
# 5) SPLIT EDGES AND VERTICES


class Polygon:
    def __init__(self, **kwargs) -> None:

        self.vertices = None
        self.edges = None
        self.rotation = None
        self.L = None
        self.convex_type = None
        self.simple_type = None
        self.triangluation = None
        self.split_edges = None
        self.split_vertices = None
        self.color = "black"
        self.__dict__.update(kwargs)

        assert self.vertices or self.edges, "Polygon: vertices and edges are empty"

        if self.edges is None:
            self.edges = self._edges()

        if self.vertices is None:
            self.vertices = self._vertices()

        if self.L is None:
            self.L = len(self.vertices)

        if self.convex_type is None:
            self.convex_type = self._convex_type()

        if self.simple_type is None:
            self.simple_type, self._intersection_pairs = self._simple_type()

        if self.split_edges is None:
            self.split_edges, self.split_vertices = self._split_edges_and_vertices()

        if self.rotation is None:
            self.rotation = self._rotation()

        if self.triangluation is None:
            self.triangluation = self._triangluation()

        self.color_rgb = ImageColor.getrgb(self.color)

    def _edges(self):
        return [
            Edge(p1, p2)
            for p1, p2 in zip(
                self.vertices, self.vertices[1:] + [self.vertices[0]]
            )
        ]

    def _vertices(self):
        return [edge[0] for edge in self.edges]

    def _convex_type(self):
        flag = None
        for i in range(self.L):
            for j in range(i + 1, i + 1 + self.L - 2):
                cl_type = classifyEdgePoint(self.edges[j % self.L], self.vertices[i])
                if flag is None:
                    flag = cl_type
                elif flag != cl_type:
                    return "CONCAVE"
        return "CONVEX"

    def _simple_type(self):
        flag = "SIMPLE"
        intersection_pairs = {}
        for i in range((self.L + 1) // 2):
            for j in range(i + 1, i + 1 + self.L - 2):
                jmod = j % self.L
                vi, vj = self.edges[i], self.edges[jmod]
                cross_type, ti = crossEdges(vi, vj)
                if cross_type == "SKEW_CROSS":
                    intersection_point = vi.a + (vi.b - vi.a) * ti
                    tj = (intersection_point - vj.a).norm() / vj.v.norm()
                    intersection_pairs[(i, jmod)] = (intersection_point, ti, tj)
                    flag = "SELF_INTERSECTED"
        return flag, intersection_pairs

    def _rotation(self):
        signed_area = self.signed_area()
        if signed_area > 0:
            return "CCW"
        return "CW"

    def _split_edges_and_vertices(self):
        if self.simple_type == "SIMPLE":
            return [self.edges], [self.vertices]
        new_edges = {}
        t_s = {}
        for k, v in self._intersection_pairs.items():
            intersection_point, ti, tj = v
            i, j = k
            for idx, t in ((i, ti), (j, tj)):
                if idx not in new_edges:
                    new_edges[idx] = [
                        Edge(self.edges[idx].a, intersection_point),
                        Edge(intersection_point, self.edges[idx].b),
                    ]
                    t_s[idx] = [t]
                else:
                    split_index = next((x[0] for x in enumerate(t_s[idx]) if x[1] > t), len(t_s[idx]))
                    t_s[idx].insert(split_index, t)
                    edge2split = new_edges[idx].pop(split_index)
                    new_edges[idx].insert(
                        split_index, Edge(intersection_point, edge2split.b)
                    )
                    new_edges[idx].insert(
                        split_index, Edge(edge2split.a, intersection_point)
                    )
        return new_edges, 1

    def signed_area(self):
        signed_area = 0.5 * sum(np.cross(edge.a.xyz, edge.b.xyz) for edge in self.split_edges[0])
        return signed_area

    def EO(self, p):
        intersection_sum = 0
        for edge in self.edges:
            match classifyEdgePoint(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    intersection_sum += 1
                case "CROSS_RIGHT":
                    intersection_sum += 1
        if intersection_sum % 2 == 1:
            return "INSIDE"
        return "OUTSIDE"

    def NZW(self, p):
        winding_number = 0
        for edge in self.edges:
            match classifyEdgePoint(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    winding_number += 1
                case "CROSS_RIGHT":
                    winding_number -= 1
        if winding_number == 0:
            return "OUTSIDE"
        return "INSIDE"

    def _triangluation(self):
        return 1
    #     # return adjacency matrix?
    #     all_points = self.vertices
    #     for edge in self.edges:
    #         all_points.extend(edge.intersection_points())
    #     all_points = sorted(all_points, lambda x: x.xyz[0])
    #     triangualtion = Polygon(*all_points[:3], color=tricolor)
    #     for point in all_points[3:]:
    #         for edge in triangualtion:
    #             view_dir = Vector(edge[0] - point)
    #         poly = Polygon(
    #             edges=triangualtion.edges+[Edge(v, point) for v in triangualtion.vertices],
    #             color=tricolor
    #         )

In [297]:
def edge_visible(): ...

In [298]:
def check_edge_visibility(face, view_direction=np.array([0, 0, 1000])):
    v1, v2, v3, *_ = face
    normal = np.cross(np.array([*(v3 - v1)]), np.array([*(v2 - v1)]))
    dot_product = np.dot(normal, view_direction)
    return dot_product <= 0

In [299]:
def draw_polygon(polygon, draw):
    for edge in polygon.edges:
        draw.point(pixelEdge(edge), fill=polygon.color)

In [300]:
def pixelLine(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    ix = 1 if dx > 0 else -1
    iy = 1 if dy > 0 else -1
    dx *= ix
    dy *= iy
    x = x1
    y = y1
    pixels = []
    if dx < dy:
        E = 2 * dx - dy
        for _ in range(dy + 1):
            pixels.append((x, y))
            if E >= 0:
                x += ix
                E -= 2 * dy
            y += iy
            E += 2 * dx
    else:
        E = 2 * dy - dx
        for _ in range(dx + 1):
            pixels.append((x, y))
            if E >= 0:
                y += iy
                E -= 2 * dx
            x += ix
            E += 2 * dy
    return pixels


def pixelEdge(e):
    x1, y1, x2, y2 = (
        round(e.a.xyz[0]),
        round(e.a.xyz[1]),
        round(e.b.xyz[0]),
        round(e.b.xyz[1]),
    )
    return pixelLine(x1, y1, x2, y2)

In [301]:
def classify(x1, y1, x2, y2, x, y):
    ax = x2 - x1
    ay = y2 - y1
    bx = x - x1
    by = y - y1
    s = ax * by - ay * bx
    if s > 0:
        return "LEFT"
    if s < 0:
        return "RIGHT"
    if ax * bx < 0 or ay * by < 0:
        return "BEHIND"
    if ax**2 + ay**2 < bx**2 + by**2:
        return "INFRONT"
    if x == x1 and y == y1:
        return "ORIGIN"
    if x == x2 and y == y2:
        return "DESTINATION"
    return "BETWEEN"


def classifyEdgePoint(e, p):
    return classify(*e.a.xyz, *e.b.xyz, *p.xyz)

In [302]:
def intersect(ax, ay, bx, by, cx, cy, dx, dy):
    nx = dy - cy
    ny = cx - dx
    denom = nx * (bx - ax) + ny * (by - ay)
    if denom == 0:
        type = classify(cx, cy, dx, dy, ax, ay)
        if type == "LEFT" or type == "RIGHT":
            return "PARALLEL", None
        else:
            return "SAME", None
    num = nx * (ax - cx) + ny * (ay - cy)
    t = -num / denom
    return "SKEW", t


def intersectEdges(e1, e2):
    return intersect(*e1.a.xyz, *e1.b.xyz, *e2.a.xyz, *e2.b.xyz)

In [303]:
def cross(ax, ay, bx, by, cx, cy, dx, dy):
    type, tab = intersect(ax, ay, bx, by, cx, cy, dx, dy)
    if type == "SAME" or type == "PARALLEL" or tab is None:
        return type, None
    if tab < 0 or tab > 1:
        return "SKEW_NO_CROSS", None
    _, tcd = intersect(cx, cy, dx, dy, ax, ay, bx, by)
    if tcd is not None and (tcd < 0 or tcd > 1):
        return "SKEW_NO_CROSS", None
    return "SKEW_CROSS", tab


def crossEdges(e1, e2):
    return cross(*e1.a.xyz, *e1.b.xyz, *e2.a.xyz, *e2.b.xyz)

In [304]:
img = Image.new("RGB", (200, 200), "white")
draw = ImageDraw.Draw(img)

In [305]:
p1 = Polygon(
    vertices=[
        Point(10, 10),
        Point(40, 100),
        Point(100, 150),
        Point(50, 180),
        Point(70, 10),
    ],
    color="green",
)

In [306]:
p1.convex_type

'CONCAVE'

In [307]:
p1.simple_type

'SELF_INTERSECTED'

In [308]:
draw_polygon(p1, draw)

In [309]:
p2 = Polygon(
    vertices=[
        Point(20, 10),
        Point(50, 10),
        Point(80, 20),
        Point(90, 50),
        Point(70, 80),
        Point(40, 90),
        Point(10, 60),
    ],
    color="yellow",
)

In [310]:
p2.convex_type

'CONVEX'

In [311]:
p2.simple_type

'SELF_INTERSECTED'

In [312]:
draw_polygon(p2, draw)
# img.show()

In [313]:
triangle = Polygon(
    vertices=[
        Point(34, 80), 
        Point(57, 1), 
        Point(170, 80),
        ], 
        color="yellow"  # A B  # B C  # C A
)

In [314]:
def paint_triangle():
    edges = [pixelEdge(edge) for edge in triangle.edges]
    ABC = edges[0] + edges[1]
    i = 0
    AC = edges[2][::-1]
    j = 0
    A = AC[0]
    C = AC[-1]
    dxAC = abs(A[0] - C[0])
    dyAC = abs(A[1] - C[1])
    k = 0 if dxAC > dyAC else 1
    pixels = []
    while ABC[i] != C:
        line = pixelLine(*ABC[i], *AC[j])
        pixels.extend(line)
        i += 1
        if AC[j][k] != ABC[i][k]:
            j += 1
    return pixels

In [315]:
draw.point(paint_triangle(), fill="red")
img.show()